# NEO and Elastic Inference
1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
3. [Training the model](#Training-the-model)
4. [Inference with the vanilla model](#Inference-with-the-vanilla-model)
5. [Inference with optimized model](#Inference-with-the-optimized-model)

## Introduction

Welcome to our model optimization example for image classification.

In this demo, we will use the Amazon sagemaker image classification algorithm to train on the [caltech-256 dataset](http://www.vision.caltech.edu/Image_Datasets/Caltech256/) and then we will demonstrate Amazon Sagemaker Neo's ability to optimize models.


To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on.

## Prequisites and Preprocessing

### Permissions and environment variables

Here we set up the linkage and authentication to AWS services. There are three parts to this:

* The roles used to give learning and hosting access to your data. This will automatically be obtained from the role used to start the notebook
* The S3 bucket that you want to use for training and model data
* The Amazon sagemaker image classification docker image which need not be changed

In [76]:
from __future__ import print_function

import sagemaker
from sagemaker import get_execution_role
import mxnet as mx

import mxnet.ndarray as nd
from mxnet import nd, autograd, gluon
from mxnet.gluon.data.vision import transforms

import numpy as np
from tqdm import tqdm

role = get_execution_role()
sess = sagemaker.Session()
bucket=sess.default_bucket()
prefix = 'ic-neo'

wget http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar
tar -xf 256_ObjectCategories.tar

In [77]:
from sagemaker.amazon.amazon_estimator import get_image_uri

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

### Data preparation
Download the data and transfer to S3 for use in training. In this demo, we are using [Caltech-256](http://www.vision.caltech.edu/Image_Datasets/Caltech256/) dataset, which contains 30608 images of 256 objects. For the training and validation data, we follow the splitting scheme in this MXNet [example](https://github.com/apache/incubator-mxnet/blob/master/example/image-classification/data/caltech256.sh). In particular, it randomly selects 60 images per class for training, and uses the remaining data for validation. The algorithm takes `RecordIO` file as input. The user can also provide the image files as input, which will be converted into `RecordIO` format using MXNet's [im2rec](https://mxnet.incubator.apache.org/how_to/recordio.html?highlight=im2rec) tool. It takes around 50 seconds to convert the entire Caltech-256 dataset (~1.2GB) on a p2.xlarge instance. However, for this demo, we will use record io format. 

In [78]:
import os 
import urllib.request
import boto3

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

        
def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)


# caltech-256
#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')
#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')

In [35]:
#tree = os.walk('/home/ec2-user/SageMaker')
#tree.
# paths = []
# dirss = []
# filess = []
# for path, dirs, files in os.walk('/home/ec2-user/SageMaker/256_ObjectCategories/', followlinks=True):
#     paths.append(path)
#     dirss.append(dirs)
#     filess.append(files)

In [105]:
from glob import glob

dirs = os.listdir('/home/ec2-user/SageMaker/256_ObjectCategories/')
dir_dict = {}
for d in dirs:
    files = glob(f'/home/ec2-user/SageMaker/256_ObjectCategories/{d}/*jpg')
    files.sort()
    dir_dict[d] = files

In [54]:
#!cd .. && mkdir train1_256
#!cd .. && mkdir train2_256

In [116]:
for d in tqdm(dir_dict):
    for path in dir_dict[d][:30]:
        try:
            os.mkdir(f'/home/ec2-user/SageMaker/train1_256/{d}')
        except:
            print(f'path: /home/ec2-user/SageMaker/train1_256/{d} already exists!')
        os.system(f'cp {path} /home/ec2-user/SageMaker/train1_256/{d}')
    
    for path in dir_dict[d][30:60]:
        try:
            os.mkdir(f'/home/ec2-user/SageMaker/train2_256/{d}')
        except:
            print(f'path: /home/ec2-user/SageMaker/train2_256/{d} already exists!')
        os.system(f'cp {path} /home/ec2-user/SageMaker/train2_256/{d}')


  0%|          | 0/257 [00:00<?, ?it/s]

path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train1_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/tra

  5%|▌         | 14/257 [00:00<00:05, 41.06it/s]

path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/train2_256/126.ladder already exists!
path: /home/ec2-user/SageMaker/tra

 28%|██▊       | 72/257 [00:00<00:03, 53.89it/s]

...

path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/240.watch-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/129.leopards-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/129.leopards-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/129.leopards-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/129.leopards-

 87%|████████▋ | 223/257 [00:04<00:01, 27.19it/s]

path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train1_256/090.gorilla already exists!
path: /home/ec2-user/SageMaker/train2_256/090.gorilla already exists!
path: /home/ec2-user

100%|██████████| 257/257 [00:04<00:00, 58.21it/s]

path: /home/ec2-user/SageMaker/train2_256/253.faces-easy-101 already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!
path: /home/ec2-user/SageMaker/train1_256/138.mattress already exists!


In [ ]:
# need to split into multiple train files 

In [98]:
#!python im2rec.py --help

In [117]:
#%%sh

# wget http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar
# tar -xf 256_ObjectCategories.tar
# cd ..

# mkdir -p caltech_256_train_60
# for i in 256_ObjectCategories/*; do
#     c=`basename $i`
#     mkdir -p caltech_256_train_60/$c
#     for j in `ls $i/*.jpg | shuf | head -n 60`; do
#         mv $j caltech_256_train_60/$c/
#     done
# done

# python distributed-train-and-deploy/im2rec.py --list --recursive caltech-256-60-train caltech_256_train_60/
# python distributed-train-and-deploy/im2rec.py --list --recursive caltech-256-60-val 256_ObjectCategories/
# python distributed-train-and-deploy/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-60-val 256_ObjectCategories/
# python distributed-train-and-deploy/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-60-train caltech_256_train_60/

In [100]:
# /home/ec2-user/SageMaker/
# /home/ec2-user/SageMaker/
#!cd .. && python distributed-train-and-deploy/im2rec.py --list True --recursive True caltech-256-train1 train1_256/


In [118]:
!cd .. && python distributed-train-and-deploy/im2rec.py --list --recursive caltech-256-train1 train1_256/
!cd .. && python distributed-train-and-deploy/im2rec.py --list --recursive caltech-256-train2 train2_256/
!cd .. && python distributed-train-and-deploy/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-train1 train1_256/
!cd .. && python distributed-train-and-deploy/im2rec.py --resize 256 --quality 90 --num-thread 16 caltech-256-train2 train2_256/


. 0
001.ak47 1
002.american-flag 2
003.backpack 3
004.baseball-bat 4
005.baseball-glove 5
006.basketball-hoop 6
007.bat 7
008.bathtub 8
009.bear 9
010.beer-mug 10
011.billiards 11
012.binoculars 12
013.birdbath 13
014.blimp 14
015.bonsai-101 15
016.boom-box 16
017.bowling-ball 17
018.bowling-pin 18
019.boxing-glove 19
020.brain-101 20
021.breadmaker 21
022.buddha-101 22
023.bulldozer 23
024.butterfly 24
025.cactus 25
026.cake 26
027.calculator 27
028.camel 28
029.cannon 29
030.canoe 30
031.car-tire 31
032.cartman 32
033.cd 33
034.centipede 34
035.cereal-box 35
036.chandelier-101 36
037.chess-board 37
038.chimp 38
039.chopsticks 39
040.cockroach 40
041.coffee-mug 41
042.coffin 42
043.coin 43
044.comet 44
045.computer-keyboard 45
046.computer-monitor 46
047.computer-mouse 47
048.conch 48
049.cormorant 49
050.covered-wagon 50
051.cowboy-hat 51
052.crab-101 52
053.desk-globe 53
054.diamond-ring 54
055.dice 55
056.dog 56
057.dolphin-101 57
058.doorknob 58
059.drinking-straw 59
060.duck 60
0

Creating .rec file from /home/ec2-user/SageMaker/caltech-256-train1.lst in /home/ec2-user/SageMaker
time: 0.009346961975097656  count: 0
time: 0.6158983707427979  count: 1000
time: 0.3462412357330322  count: 2000
time: 0.36517333984375  count: 3000
time: 0.4259769916534424  count: 4000
time: 0.4094071388244629  count: 5000
time: 0.3123610019683838  count: 6000
time: 0.3035609722137451  count: 7000
time: 0.33539891242980957  count: 8000
Creating .rec file from /home/ec2-user/SageMaker/caltech-256-train2.lst in /home/ec2-user/SageMaker
time: 0.0050737857818603516  count: 0
time: 0.4672091007232666  count: 1000
time: 0.3819105625152588  count: 2000
time: 0.35790228843688965  count: 3000
time: 0.3912510871887207  count: 4000
time: 0.38599514961242676  count: 5000
time: 0.43807387351989746  count: 6000
time: 0.2687242031097412  count: 7000


In [17]:
# record = mx.recordio.MXRecordIO('caltech-256-60-train.rec', 'r')
# item = record.read()
# header, img = mx.recordio.unpack_img(item)

In [119]:
# Four channels: train, validation, train_lst, and validation_lst
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)

# upload the lst files to train and validation channels
!cd .. && aws s3 cp caltech-256-train1.rec $s3train --quiet
!cd .. && aws s3 cp caltech-256-train2.rec $s3train --quiet
!cd .. && aws s3 cp caltech-256-train1.rec $s3validation --quiet




Once we have the data available in the correct format for training, the next step is to actually train the model using the data. After setting training parameters, we kick off training, and poll for status until training is completed.


## Training the model

Now that we are done with all the setup that is needed, we are ready to train our object detector. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.
### Training parameters
There are two kinds of parameters that need to be set for training. The first one are the parameters for the training job. These include:

* **Training instance count**: This is the number of instances on which to run the training. When the number of instances is greater than one, then the image classification algorithm will run in distributed settings. ml.p3.8xlarge has 4 x Nvidia V100.  https://aws.amazon.com/sagemaker/pricing/instance-types/
* **Training instance type**: This indicates the type of machine on which to run the training. Typically, we use GPU instances for these training 
* **Output path**: This the s3 folder in which the training output is stored


In [112]:
s3_output_location = 's3://{}/{}/output-ic'.format(bucket, prefix)
ic = sagemaker.estimator.Estimator(training_image,
                                         role, 
                                         train_instance_count=2, 
                                         train_instance_type='ml.p3.2xlarge', 
                                         train_volume_size = 50,
                                         train_max_run = 360000,
                                         input_mode= 'Pipe',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

Apart from the above set of parameters, there are hyperparameters that are specific to the algorithm. These are:

https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html

* **num_layers**: The number of layers (depth) for the network. Valid values: For data with large image size (224x224) positive integer in [18, 34, 50, 101, 152, 200] or for data with small image size (28x28) use [20, 32, 44, 56, 110]
* **image_shape**: The input image dimensions,'num_channels, height, width', for the network. It should be no larger than the actual image size. The number of channels should be same as the actual image.
* **num_classes**: This is the number of output classes for the new dataset. Imagenet was trained with 1000 output classes but the number of output classes can be changed for fine-tuning. For caltech, we use 257 because it has 256 object categories + 1 clutter class.
* **num_training_samples**: This is the total number of training samples. It is set to 15240 for caltech dataset with the current split.
* **mini_batch_size**: The number of training samples used for each mini batch. In distributed training, the number of training samples used per batch will be N * mini_batch_size where N is the number of hosts on which training is run.
* **epochs**: Number of training epochs.
* **learning_rate**: Learning rate for training.
* **top_k**: Report the top-k accuracy during training.
* **precision_dtype**: Training datatype precision (default: float32). If set to 'float16', the training will be done in mixed_precision mode and will be faster than float32 mode
* **use_pretrained_model**:  Flag to use pre-trained model for training. If set to 1, then the pretrained model with the corresponding number of layers is loaded and used for training. Only the top FC layer are reinitialized with random weights. Otherwise, the network is trained from scratch.


In [113]:
ic.set_hyperparameters(num_layers=101,
                             image_shape = "3,224,224",
                             num_classes=257,
                             num_training_samples=15420,
                             mini_batch_size=128,
                             epochs=2,
                             learning_rate=0.0002,
                             top_k=2,
                             precision_dtype='float32',
                             use_pretrained_model=1)

## Input data specification
Set the data type and channels used for training

In [114]:

# change to shardbys3
train_data = sagemaker.session.s3_input(s3train, distribution='ShardedByS3Key', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data}

## Start the training
Start training by calling the fit method in the estimator

In [115]:
ic.fit(inputs=data_channels, logs=True)

2020-05-27 14:20:40 Starting - Starting the training job...
2020-05-27 14:20:41 Starting - Launching requested ML instances......
2020-05-27 14:21:57 Starting - Preparing the instances for training.........
2020-05-27 14:23:34 Downloading - Downloading input data
2020-05-27 14:23:34 Training - Downloading the training image....Docker entrypoint called with argument(s): train
[05/27/2020 14:24:15 INFO 140112449828672] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[05/27/2020 14:24:15 INFO 140112449828672] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {u'lear


2020-05-27 14:24:22 Uploading - Uploading generated training model
2020-05-27 14:24:22 Failed - Training job failed


UnexpectedStatusException: Error for Training job image-classification-2020-05-27-14-20-39-752: Failed. Reason: ClientError: train channel S3DistributionType should be FullyReplicated.

# Inference with the vanilla model

***

Now we will test the trained model without any specific optimization for the hardware.

In [ ]:
ic_vanilla = ic.deploy(initial_instance_count = 1,
                          instance_type = 'ml.p3.2xlarge')

### Download test image

In [ ]:
!wget -O /tmp/test-1.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/080.frog/080_0001.jpg
!wget -O /tmp/test-2.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/002.american-flag/002_0004.jpg
!wget -O /tmp/test-3.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/037.chess-board/037_0006.jpg
!wget -O /tmp/test-4.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/086.golden-gate-bridge/086_0010.jpg

### Evaluation

Evaluate the image through the network for inteference. The network outputs class probabilities and typically, one selects the class with the maximum probability as the final class output.

**Note:** The output class detected by the network may not be accurate in this example. To limit the time taken and cost of training, we have trained the model only for 5 epochs. If the network is trained for more epochs (say 20), then the output class will be more accurate.

In [ ]:
#file_name = '/tmp/test-1.jpg'
file_name = '/tmp/test-2.jpg'
#file_name = '/tmp/test-3.jpg'
#file_name = '/tmp/test-4.jpg'
# test image
from IPython.display import Image
Image(file_name)  

In [ ]:
import json
import numpy as np

with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
    
ic_vanilla.content_type = 'application/x-image'

**Time the prediction with the vanilla model.**

In [ ]:
result = json.loads(ic_vanilla.predict(payload))

In [ ]:
# the result will output the probabilities for all classes
# find the class with maximum probability and print the class index
index = np.argmax(result)
object_categories = ['ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'basketball-hoop', 'bat', 'bathtub', 'bear', 'beer-mug', 'billiards', 'binoculars', 'birdbath', 'blimp', 'bonsai-101', 'boom-box', 'bowling-ball', 'bowling-pin', 'boxing-glove', 'brain-101', 'breadmaker', 'buddha-101', 'bulldozer', 'butterfly', 'cactus', 'cake', 'calculator', 'camel', 'cannon', 'canoe', 'car-tire', 'cartman', 'cd', 'centipede', 'cereal-box', 'chandelier-101', 'chess-board', 'chimp', 'chopsticks', 'cockroach', 'coffee-mug', 'coffin', 'coin', 'comet', 'computer-keyboard', 'computer-monitor', 'computer-mouse', 'conch', 'cormorant', 'covered-wagon', 'cowboy-hat', 'crab-101', 'desk-globe', 'diamond-ring', 'dice', 'dog', 'dolphin-101', 'doorknob', 'drinking-straw', 'duck', 'dumb-bell', 'eiffel-tower', 'electric-guitar-101', 'elephant-101', 'elk', 'ewer-101', 'eyeglasses', 'fern', 'fighter-jet', 'fire-extinguisher', 'fire-hydrant', 'fire-truck', 'fireworks', 'flashlight', 'floppy-disk', 'football-helmet', 'french-horn', 'fried-egg', 'frisbee', 'frog', 'frying-pan', 'galaxy', 'gas-pump', 'giraffe', 'goat', 'golden-gate-bridge', 'goldfish', 'golf-ball', 'goose', 'gorilla', 'grand-piano-101', 'grapes', 'grasshopper', 'guitar-pick', 'hamburger', 'hammock', 'harmonica', 'harp', 'harpsichord', 'hawksbill-101', 'head-phones', 'helicopter-101', 'hibiscus', 'homer-simpson', 'horse', 'horseshoe-crab', 'hot-air-balloon', 'hot-dog', 'hot-tub', 'hourglass', 'house-fly', 'human-skeleton', 'hummingbird', 'ibis-101', 'ice-cream-cone', 'iguana', 'ipod', 'iris', 'jesus-christ', 'joy-stick', 'kangaroo-101', 'kayak', 'ketch-101', 'killer-whale', 'knife', 'ladder', 'laptop-101', 'lathe', 'leopards-101', 'license-plate', 'lightbulb', 'light-house', 'lightning', 'llama-101', 'mailbox', 'mandolin', 'mars', 'mattress', 'megaphone', 'menorah-101', 'microscope', 'microwave', 'minaret', 'minotaur', 'motorbikes-101', 'mountain-bike', 'mushroom', 'mussels', 'necktie', 'octopus', 'ostrich', 'owl', 'palm-pilot', 'palm-tree', 'paperclip', 'paper-shredder', 'pci-card', 'penguin', 'people', 'pez-dispenser', 'photocopier', 'picnic-table', 'playing-card', 'porcupine', 'pram', 'praying-mantis', 'pyramid', 'raccoon', 'radio-telescope', 'rainbow', 'refrigerator', 'revolver-101', 'rifle', 'rotary-phone', 'roulette-wheel', 'saddle', 'saturn', 'school-bus', 'scorpion-101', 'screwdriver', 'segway', 'self-propelled-lawn-mower', 'sextant', 'sheet-music', 'skateboard', 'skunk', 'skyscraper', 'smokestack', 'snail', 'snake', 'sneaker', 'snowmobile', 'soccer-ball', 'socks', 'soda-can', 'spaghetti', 'speed-boat', 'spider', 'spoon', 'stained-glass', 'starfish-101', 'steering-wheel', 'stirrups', 'sunflower-101', 'superman', 'sushi', 'swan', 'swiss-army-knife', 'sword', 'syringe', 'tambourine', 'teapot', 'teddy-bear', 'teepee', 'telephone-box', 'tennis-ball', 'tennis-court', 'tennis-racket', 'theodolite', 'toaster', 'tomato', 'tombstone', 'top-hat', 'touring-bike', 'tower-pisa', 'traffic-light', 'treadmill', 'triceratops', 'tricycle', 'trilobite-101', 'tripod', 't-shirt', 'tuning-fork', 'tweezer', 'umbrella-101', 'unicorn', 'vcr', 'video-projector', 'washing-machine', 'watch-101', 'waterfall', 'watermelon', 'welding-mask', 'wheelbarrow', 'windmill', 'wine-bottle', 'xylophone', 'yarmulke', 'yo-yo', 'zebra', 'airplanes-101', 'car-side-101', 'faces-easy-101', 'greyhound', 'tennis-shoes', 'toad', 'clutter']
print("Result: label - " + object_categories[index] + ", probability - " + str(result[index]))

## Inference with the Elastic Compute

In [ ]:
from sagemaker import Model

In [ ]:
ic_model = Model(model_data="s3://sagemaker-us-east-2-425479682919/ic/output/image-classification-2020-05-06-16-50-22-338/output/model.tar.gz",
                    role=role,
                    image=training_image)

In [ ]:
ic_eia = ic_model.deploy(initial_instance_count = 1,
                          instance_type = 'ml.c5.large',
                          accelerator_type='ml.eia2.medium')

In [ ]:
result = json.loads(ic_eia.predict(payload))

# Inference with the optimized model

***

We will use Sagemaker Neo to optimize the model.

## Introduction to SageMaker Neo

***

[Amazon SageMaker Neo](https://aws.amazon.com/sagemaker/neo/) optimizes models to run up to fourth as fast, with less than a tenth of the memory footprint, with no loss in accuracy. You start with a machine learning model built using MXNet, TensorFlow, PyTorch, or XGBoost and trained using Amazon SageMaker. Then you choose your target hardware platform from Intel, NVIDIA, or ARM. With a single click, SageMaker Neo will then compile the trained model into an executable. In this example, we will use the model we just trained and see how well the optimized model could perform.

## Optimize the model specifically for the architecture using Neo API
Now we will compare the same model, but compiled specifically for the architecture we're deploying on.

In [ ]:
output_path = '/'.join(ic.output_path.split('/')[:-1])

optimized_ic = ic.compile_model(target_instance_family='ml_p3', 
                                input_shape={'data':[1, 3, 224, 224]},  # Batch size 1, 3 channels, 224x224 Images.
                                output_path=output_path,
                                framework='mxnet', framework_version='gpu')

In [ ]:
optimized_ic.image = get_image_uri(sess.boto_region_name, 'image-classification-neo', repo_version="gpu")

optimized_ic.name = 'deployed-image-classification-neo'

## Deploy optimized model

In [ ]:
optimized_ic_classifier = optimized_ic.deploy(initial_instance_count = 1,
                                              instance_type = 'ml.p3.2xlarge')

## Make predictions

In [ ]:
optimized_ic_classifier.content_type = 'application/x-image'

**Time the prediction with the optimized model.** We compare the optimized predictions times to the vanilla predictions times.

In [ ]:
result = json.loads(optimized_ic_classifier.predict(payload))

In [ ]:
result = json.loads(ic_classifier.predict(payload))

In [ ]:
# the result will output the probabilities for all classes
# find the class with maximum probability and print the class index
index = np.argmax(result)
print("Result: label - " + object_categories[index] + ", probability - " + str(result[index]))

## Clean up


When we're done with the endpoint, we can just delete it and the backing instances will be released. Uncomment and run the following cell to delete the endpoint and model

In [ ]:
ic_eia.delete_endpoint()
ic_vanilla.delete_endpoint()
optimized_ic_classifier.delete_endpoint()

ls: cannot access /home/ec2-user/mxnet: No such file or directory


In [22]:
!java --version

Unrecognized option: --version
Error: Could not create the Java Virtual Machine.
Error: A fatal exception has occurred. Program will exit.
